In [3]:
from llama_cpp import Llama

llm = Llama(model_path="mistral-7b-instruct-v0.2.Q4_K_M.gguf")
response = llm("描述一個美好夏夜場景")
print(response["choices"][0]["text"])


llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 l

：

A beautiful summer night scene is painted with the soft glow of the


In [25]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms.base import LLM
from llama_cpp import Llama
from pydantic import PrivateAttr

class LlamaCppLLM(LLM):
    _llm: Llama = PrivateAttr()

    temperature: float = 0.5
    max_tokens: int = 2048

    def __init__(self, model_path: str, **kwargs):
        super().__init__(**kwargs)
        self._llm = Llama(model_path=model_path, n_ctx=4096)


    @property
    def _llm_type(self):
        return "llama_cpp"

    def _call(self, prompt, stop=None):
        output = self._llm(prompt, temperature=self.temperature, max_tokens=self.max_tokens)
        return output['choices'][0]['text'].strip()


model_path = "mistral-7b-instruct-v0.2.Q4_K_M.gguf"
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"

embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)

vectorstore = FAISS.load_local(
    folder_path="faiss_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

llm = LlamaCppLLM(model_path=model_path, temperature=0.5, max_tokens=2048)


template = """根據以下提供的資訊，回答問題。
資訊:
{context}

問題: {question}
答案:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 1}),  # 限制回傳3個chunks
    chain_type_kwargs={"prompt": prompt}
)

if __name__ == "__main__":
    query = "請描述一個美好的夏夜。"
    answer = qa_chain.run(query)
    print("Q:", query)
    print("A:", answer)


llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 l

Q: 請描述一個美好的夏夜。
A: 一個美好的夏夜可能會像 Sylvia Plath 的詩歌 "Ariel" 所描述的早上騎馬向日輝的景象一樣，只是換成晚上，景象更加美麗而愉悅。你可能會看到紅色的樹葉在綠色的地上閃閃发光，晚霾中漫游的蟲群，天空中燦燦發光的星星，和蝴蝶在空旷的花園中飛舞。在這種美好的夏夜中，你可能會聽到燒燒的燈火和舒服而溫柔的風，感受到濕濕的草地和熱涼的空气。這是一個充滿美好情感和愉悅感的夏夜，令人心靈饒了。
